# Imports

In [90]:
import pandas as pd
import numpy as np
import keras
import math

# Load the dataset

In [185]:
# one hot encode the dataset, only take 6:00, 12:00 and 18:00 b/c that when most 
# of the times are taken
dataframe = pd.read_csv("atlantic_without_NW.csv")
dataframe["time"] = dataframe["time"].astype('category')
dataframe['date'] = [str(i)[4:6] for i in dataframe['date']] # Get just the month

dataframe = pd.get_dummies(dataframe, columns=["time","rating"])
dataframe = pd.get_dummies(dataframe, columns=["date"])
dataframe.drop(["ocean","name"],axis=1, inplace=True)

y = dataframe[["rating_ DB","rating_ EX","rating_ HU","rating_ LO","rating_ SD","rating_ SS","rating_ TD","rating_ TS","rating_ V"]].values

dataframe.drop(["rating_ DB","rating_ EX","rating_ HU","rating_ LO","rating_ SD","rating_ SS","rating_ TD","rating_ TS","rating_ V"],axis=1, inplace=True)
X = dataframe[["date","lat","long","wind_speed","pressure","time_600","time_1200","time_1800"]].values


KeyError: "['date'] not in index"


# Clean the data

In [160]:
# delete all the rows that contain no readings for pressure
rows_to_delete = []
encoded_lat_long = []
row_num = 0
for row in X:
    if row[4] == -999:
        rows_to_delete.append(row_num)
    row_num += 1

    x = (math.cos(row[1]) * math.cos(row[2]))
    y = (math.cos(row[1]) * math.sin(row[2]))
    z = math.sin(row[1])
    encoded_lat_long.append([x,y,z])
    
X = np.append(X, encoded_lat_long, axis=1) # add the new coordinates mapping
X = np.delete(X, 1, 1)                     # remove old lat
X = np.delete(X, 1, 1)                     # remove old long
X = np.delete(X, rows_to_delete, 0)        # remove all rows with bad data
y = np.delete(y, rows_to_delete, 0)


ValueError: DataFrame constructor not properly called!

# Split the dataset into training set and test set

In [56]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Train the model on training set

In [57]:
from keras.models import Sequential
from keras.layers import Dense

In [58]:
classifier = Sequential()

# input layer
classifier.add(Dense(output_dim=6, init='uniform', activation='relu', input_dim=8))

# hidden layer
classifier.add(Dense(output_dim=7, init='uniform', activation='relu'))

# hidden layer
classifier.add(Dense(output_dim=12, init='uniform', activation='relu'))

# output layer
classifier.add(Dense(output_dim=9, init='uniform', activation='softmax'))

# create the NN
classifier.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# Predict the test set


In [59]:

classifier.fit(x=X_train, y=y_train, validation_data=(X_test,y_test), batch_size=10, epochs=200)

Train on 12380 samples, validate on 3095 samples
Epoch 1/200
12380/12380 [==============================] - 4s 292us/step - loss: 3.2164 - accuracy: 0.3163 - val_loss: 1.6698 - val_accuracy: 0.3405
Epoch 2/200
12380/12380 [==============================] - 4s 287us/step - loss: 1.6469 - accuracy: 0.3384 - val_loss: 1.6617 - val_accuracy: 0.3405
Epoch 3/200
12380/12380 [==============================] - 4s 303us/step - loss: 1.6469 - accuracy: 0.3439 - val_loss: 1.6693 - val_accuracy: 0.3405
Epoch 4/200
12380/12380 [==============================] - 4s 297us/step - loss: 1.6524 - accuracy: 0.3446 - val_loss: 1.7259 - val_accuracy: 0.3405
Epoch 5/200
12380/12380 [==============================] - 4s 301us/step - loss: 1.6559 - accuracy: 0.3422 - val_loss: 1.6955 - val_accuracy: 0.3405
Epoch 6/200
12380/12380 [==============================] - 4s 297us/step - loss: 1.6519 - accuracy: 0.3441 - val_loss: 1.6626 - val_accuracy: 0.3405
Epoch 7/200
12380/12380 [==============================] 

KeyboardInterrupt: 

In [113]:
GH = np.array([[5,1092,0,1,0]])

y_pred = classifier.predict(GH)

In [114]:
for row in y_pred:
    most_likely = 0
    for el in row:
        if el > most_likely:
            most_likely = el

    print( "rating_DB: {DB} \nrating_EX: {EX}\nrating_HU: {HU} \nrating_LO: {LO} \nrating_SD: {SD} \nrating_SS: {SS} \nrating_TD: {TD} \nrating_TS: {TS} \nrating_WV: {WV} \n\nMost Likely: {ml}\n\n".format(DB=row[0], EX=row[1], HU=row[2], LO=row[3], SD=row[4], SS=row[5], TD=row[6], TS=row[7], WV=row[8], ml=most_likely) )

rating_DB: 0.03127315267920494 
rating_EX: 0.11299780756235123
rating_HU: 0.0 
rating_LO: 0.35213130712509155 
rating_SD: 0.031951189041137695 
rating_SS: 2.0563235901376042e-14 
rating_TD: 0.4602642059326172 
rating_TS: 1.2766701758260002e-12 
rating_WV: 0.011382324621081352 

Most Likely: 0.4602642059326172


